In [1]:
# app.py
# Drug Safety Assistant — Standalone script (not a notebook)

import os
import gradio as gr
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# ---------------------------
# MODEL NAME
# ---------------------------
model_name = "ibm-granite/granite-3.3-2b-instruct"

# ---------------------------
# SAFE DEVICE LOADING
# ---------------------------
def load_model_and_tokenizer(name):
    # Try to load with automatic device mapping (works if you have transformers accelerate + GPU)
    try:
        tokenizer = AutoTokenizer.from_pretrained(name)
        model = AutoModelForCausalLM.from_pretrained(
            name,
            device_map="auto",                # will attempt to place model on GPU if available
            torch_dtype=torch.float16         # smaller dtype where supported
        )
        return tokenizer, model
    except Exception as e:
        print("Auto device_map load failed, falling back to CPU. Reason:", e)
        tokenizer = AutoTokenizer.from_pretrained(name)
        model = AutoModelForCausalLM.from_pretrained(name, low_cpu_mem_usage=True)
        model.to("cpu")
        return tokenizer, model

# Attempt to load
try:
    tokenizer, model = load_model_and_tokenizer(model_name)
except Exception as e:
    print("Failed to load model. If the model is private, make sure you have logged in with 'huggingface-cli login' and have access.")
    raise

# ---------------------------
# MODULE 1: DRUG COMPARISON
# ---------------------------
def drug_comparison(drug: str):
    alternatives = {
        "Paracetamol": ["Ibuprofen", "Aspirin"],
        "Cetirizine": ["Loratadine", "Fexofenadine"],
        "Amoxicillin": ["Azithromycin", "Cefixime"]
    }
    drug = (drug or "").strip().title()
    if not drug:
        return "Please enter a drug name."
    if drug not in alternatives:
        return f"No alternatives found for {drug}."
    msg = f"Drug Comparison for **{drug}**\n\n"
    msg += "**Recommended alternatives (Least risk first):**\n"
    for alt in alternatives[drug]:
        msg += f"• {alt}\n"
    return msg

# ---------------------------
# MODULE 2: PRESCRIPTION ANALYZER
# ---------------------------
interaction_data = {
    ("Paracetamol", "Ibuprofen"): "Mild interaction. May cause stomach irritation.",
    ("Aspirin", "Ibuprofen"): "Severe interaction. Should not be combined.",
    ("Cetirizine", "Alcohol"): "High sedation risk."
}

def call_model_for_explanation(prompt: str, max_new_tokens: int = 150):
    # build a plain text prompt for the instruct model
    if not prompt:
        return "No prompt provided."
    input_text = (
        "You are a medical assistant AI. Explain simply and safely.\n\n"
        "User prompt:\n" + prompt + "\n\nProvide a short explanation."
    )
    inputs = tokenizer(input_text, return_tensors="pt")
    device = next(model.parameters()).device
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # generate (handle possible model.generate signature differences)
    output_ids = model.generate(**inputs, max_new_tokens=max_new_tokens, do_sample=False)
    # decode and return only the generated part
    decoded = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    # If the model echoed input, try to strip input_text
    if decoded.startswith(input_text):
        return decoded[len(input_text):].strip()
    return decoded

def prescription_analyzer(prescription_text: str):
    prescription_text = (prescription_text or "").strip()
    if not prescription_text:
        return "Please enter a prescription (comma-separated drug names)."

    drugs = [d.strip().title() for d in prescription_text.split(",") if d.strip()]
    result = "### Prescription Analysis\n\n"
    result += "Detected drugs: " + ", ".join(drugs) + "\n\n"

    found = False
    for i in range(len(drugs)):
        for j in range(i+1, len(drugs)):
            pair = (drugs[i], drugs[j])
            rpair = (drugs[j], drugs[i])
            if pair in interaction_data:
                result += f"⚠ **Interaction:** {pair[0]} + {pair[1]}\n→ {interaction_data[pair]}\n\n"
                found = True
            elif rpair in interaction_data:
                result += f"⚠ **Interaction:** {rpair[0]} + {rpair[1]}\n→ {interaction_data[rpair]}\n\n"
                found = True

    if not found:
        result += "No harmful interactions found.\n\n"

    # Ask the model for a short explanation
    try:
        ai_reasoning = call_model_for_explanation(f"Analyze this prescription: {prescription_text}", max_new_tokens=150)
        result += "\n### AI Explanation\n" + ai_reasoning
    except Exception as e:
        result += "\n### AI Explanation\n" + f"(Model call failed: {e})"

    return result

# ---------------------------
# MODULE 3: PRECAUTIONARY MODULE
# ---------------------------
precautions = {
    "Paracetamol": "Do not exceed 4g/day. Avoid alcohol.",
    "Ibuprofen": "Avoid with ulcers or kidney issues.",
    "Aspirin": "Not for children. Avoid in pregnancy.",
    "Cetirizine": "May cause drowsiness. Avoid driving."
}

def precaution(drug: str):
    drug = (drug or "").strip().title()
    if not drug:
        return "Please enter a drug name."
    if drug in precautions:
        return f"### Precautions for {drug}\n" + precautions[drug]
    else:
        return f"No precaution data found for {drug}."

# ---------------------------
# GRADIO UI
# ---------------------------
with gr.Blocks(title="Drug Safety Assistant") as app:
    gr.Markdown("# 🏥 Drug Safety Analysis System")
    gr.Markdown("Analyze interactions, compare drugs, and read precautions — powered by IBM Granite (if available).")

    with gr.Tab("Drug Comparison"):
        drug_in = gr.Textbox(label="Enter Drug Name")
        drug_out = gr.Markdown()
        gr.Button("Compare").click(drug_comparison, inputs=drug_in, outputs=drug_out)

    with gr.Tab("Prescription Analyzer"):
        pres_in = gr.Textbox(label="Enter Prescription (comma separated drugs)")
        pres_out = gr.Markdown()
        gr.Button("Analyze").click(prescription_analyzer, inputs=pres_in, outputs=pres_out)

    with gr.Tab("Precautionary Guide"):
        prec_in = gr.Textbox(label="Enter Drug Name")
        prec_out = gr.Markdown()
        gr.Button("Show Precautions").click(precaution, inputs=prec_in, outputs=prec_out)

# ---------------------------
# LAUNCH
# ---------------------------
if __name__ == "__main__":
    app.launch()



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f78a34a9ad05585ebc.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## Local Inference on GPU
Model page: https://huggingface.co/ibm-granite/granite-3.3-2b-instruct

⚠️ If the generated code snippets do not work, please open an issue on either the [model repo](https://huggingface.co/ibm-granite/granite-3.3-2b-instruct)
			and/or on [huggingface.js](https://github.com/huggingface/huggingface.js/blob/main/packages/tasks/src/model-libraries-snippets.ts) 🙏

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation", model="ibm-granite/granite-3.3-2b-instruct")
messages = [
    {"role": "user", "content": "Who are you?"},
]
pipe(messages)

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("ibm-granite/granite-3.3-2b-instruct")
model = AutoModelForCausalLM.from_pretrained("ibm-granite/granite-3.3-2b-instruct")
messages = [
    {"role": "user", "content": "Who are you?"},
]
inputs = tokenizer.apply_chat_template(
	messages,
	add_generation_prompt=True,
	tokenize=True,
	return_dict=True,
	return_tensors="pt",
).to(model.device)

outputs = model.generate(**inputs, max_new_tokens=40)
print(tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:]))